# ETL Project #

Our ETL project used Kaggle to pull in datasets for movies from a streaming platform and from the IMDB database.  We want to utilize the data to analyze if the profit of a movie has a significant basis for which the streaming websites will have the movie to stream.

In [1]:
# Import dependencies
import pandas as pd
from sqlalchemy import create_engine

### Extacting the CSV File into DataFrames ###
* The movies_metadata.csv file is from https://www.kaggle.com/kerneler/starter-movies-metadata-20a19a89-9, and it lists movie information from IMDB.
* The MoviesOnStreamingPlatforms_updated.csv file is from https://www.kaggle.com/ruchi798/movies-on-netflix-prime-video-hulu-and-disney, and it lists movies from several streaming platforms.

In [42]:
# Reading the movies_metadata.csv file into a frame
metadata = pd.read_csv('Resources/movies_metadata.csv')
metadata.head()

C:\ProgramData\Anaconda3\envs\PythonData\lib\site-packages\IPython\core\interactiveshell.py:3062: DtypeWarning: Columns (10) have mixed types.Specify dtype option on import or set low_memory=False.
  has_raised = await self.run_ast_nodes(code_ast.body, cell_name,


,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0
3,False,NaN,16000000,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",NaN,31357,tt0114885,en,Waiting to Exhale,"Cheated on, mistreated and stepped on, the wom...",...,1995-12-22,81452156.0,127.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Friends are the people who let you be yourself...,Waiting to Exhale,False,6.1,34.0
4,False,"{'id': 96871, 'name': 'Father of the Bride Col...",0,"[{'id': 35, 'name': 'Comedy'}]",NaN,11862,tt0113041,en,Father of the Bride Part II,Just when George Banks has recovered from his ...,...,1995-02-10,76578911.0,106.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Just When His World Is Back To Normal... He's ...,Father of the Bride Part II,False,5.7,173.0


In [6]:
# Retrieving CSV file and turning it into a dataframe
stream_file = "./Resources/MoviesOnStreamingPlatforms_updated.csv"
stream_df = pd.read_csv(stream_file)
stream_df.head()

,Unnamed: 0,ID,Title,Year,Age,IMDb,Rotten Tomatoes,Netflix,Hulu,Prime Video,Disney+,Type,Directors,Genres,Country,Language,Runtime
0,0,1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
1,1,2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
2,2,3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
3,3,4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
4,4,5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


### Transforming the DataFrames###

In [51]:
# Getting only the columns we're gonna use into a dataframe
metadata = metadata[['imdb_id','original_title','title','runtime','budget','revenue',\
                    'popularity','vote_average', 'vote_count']]

metadata.drop_duplicates("imdb_id", inplace=True)
metadata.index.name = "id"
metadata = metadata[metadata["budget"].apply(lambda x: x.isnumeric())]
metadata.head()

,imdb_id,original_title,title,runtime,budget,revenue,popularity,vote_average,vote_count
id,,,,,,,,,
0,tt0114709,Toy Story,Toy Story,81.0,30000000,373554033.0,21.9469,7.7,5415.0
1,tt0113497,Jumanji,Jumanji,104.0,65000000,262797249.0,17.0155,6.9,2413.0
2,tt0113228,Grumpier Old Men,Grumpier Old Men,101.0,0,0.0,11.7129,6.5,92.0
3,tt0114885,Waiting to Exhale,Waiting to Exhale,127.0,16000000,81452156.0,3.85949,6.1,34.0
4,tt0113041,Father of the Bride Part II,Father of the Bride Part II,106.0,0,76578911.0,8.38752,5.7,173.0


In [8]:
# Create a filtered dataframe from specific columns
stream_cols = ["ID",
               "Title",
               "Year",
               "Age",
               "IMDb",
               "Rotten Tomatoes",
               "Netflix",
               "Hulu",
               "Prime Video",
               "Disney+",
               "Type",
               "Directors",
               "Genres",
               "Country",
               "Language",
               "Runtime"]

stream_transformed= stream_df[stream_cols].copy()

# Rename the column headers
stream_transformed = stream_transformed.rename(columns={"ID": "id",
                                                        "Title": "title",
                                                        "Year": "year",
                                                        "Age": "age",
                                                        "IMDb": "imdb",
                                                        "Rotten Tomatoes": "rotten_tomatoes",
                                                        "Netflix": "netflix",
                                                        "Hulu": "hulu",
                                                        "Prime Video": "prime_video",
                                                        "Disney+": "disney",
                                                        "Type": "type",
                                                        "Directors": "directors",
                                                        "Genres": "genres",
                                                        "Country": "country",
                                                        "Language": "language",
                                                        "Runtime": "runtime"})

# Clean the data by dropping duplicates and setting the index
stream_transformed.drop_duplicates("id", inplace=True)
stream_transformed.set_index("id", inplace=True)

stream_transformed.head()

,title,year,age,imdb,rotten_tomatoes,netflix,hulu,prime_video,disney,type,directors,genres,country,language,runtime
id,,,,,,,,,,,,,,,
1,Inception,2010,13+,8.8,87%,1,0,0,0,0,Christopher Nolan,"Action,Adventure,Sci-Fi,Thriller","United States,United Kingdom","English,Japanese,French",148.0
2,The Matrix,1999,18+,8.7,87%,1,0,0,0,0,"Lana Wachowski,Lilly Wachowski","Action,Sci-Fi",United States,English,136.0
3,Avengers: Infinity War,2018,13+,8.5,84%,1,0,0,0,0,"Anthony Russo,Joe Russo","Action,Adventure,Sci-Fi",United States,English,149.0
4,Back to the Future,1985,7+,8.5,96%,1,0,0,0,0,Robert Zemeckis,"Adventure,Comedy,Sci-Fi",United States,English,116.0
5,"The Good, the Bad and the Ugly",1966,18+,8.8,97%,1,0,1,0,0,Sergio Leone,Western,"Italy,Spain,West Germany",Italian,161.0


In [35]:
# Checking the summary for IMDB movie database
metadata.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 45418 entries, 0 to 45465
Data columns (total 9 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   imdb_id         45417 non-null  object 
 1   original_title  45418 non-null  object 
 2   title           45414 non-null  object 
 3   runtime         45159 non-null  float64
 4   budget          45418 non-null  object 
 5   revenue         45414 non-null  float64
 6   popularity      45414 non-null  object 
 7   vote_average    45414 non-null  float64
 8   vote_count      45414 non-null  float64
dtypes: float64(4), object(5)
memory usage: 3.5+ MB


In [11]:
# Checking the summary for streamed movies database
stream_transformed.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 16744 entries, 1 to 16744
Data columns (total 15 columns):
 #   Column           Non-Null Count  Dtype  
---  ------           --------------  -----  
 0   title            16744 non-null  object 
 1   year             16744 non-null  int64  
 2   age              7354 non-null   object 
 3   imdb             16173 non-null  float64
 4   rotten_tomatoes  5158 non-null   object 
 5   netflix          16744 non-null  int64  
 6   hulu             16744 non-null  int64  
 7   prime_video      16744 non-null  int64  
 8   disney           16744 non-null  int64  
 9   type             16744 non-null  int64  
 10  directors        16018 non-null  object 
 11  genres           16469 non-null  object 
 12  country          16309 non-null  object 
 13  language         16145 non-null  object 
 14  runtime          16152 non-null  float64
dtypes: float64(2), int64(6), object(7)
memory usage: 2.0+ MB


### Create a database connection ###

In [13]:
# Import password key
from config import post_pass

In [17]:
# Connect to pgAdmin
connection_string = f'postgres:{post_pass}@localhost:5432/movies_db'
engine = create_engine(f'postgresql://{connection_string}')

* Inside pgAdmin, in the newly created database, a schema query is run to create the stream and the imdb tables

In [19]:
# Confirm tables
engine.table_names()

['stream', 'imdb']

### Loading the DataFrames into the database ###

In [23]:
# Loading the stream dataframe into the pgAdmin database in movies_db
stream_transformed.to_sql(name='stream', con=engine, if_exists='append', index=True)

In [52]:
# Loading the imdb dataframe into the pgAdmin database in movies_db
metadata.to_sql(name='imdb', con=engine, if_exists='append', index=True)